# Tests for SMM (without dask) versus CDO

There are a few tests to check if the SMM approach is faster than the CDO one and if it is reliable in terms of output. Encouraging results below. Tested with both 2D and 3D vars, using DataArray and Datasets

In [1]:
import timeit
import os
import numpy as np
import xarray as xr
from smmregrid import cdo_generate_weights, Regridder
from smmregrid.checker import check_cdo_regrid # this is a new function introduced to verify the output
from cdo import Cdo
import pandas as pd
import copy
cdo = Cdo()
import dask
dask.config.set(scheduler="synchronous")

# where and which the data are
indir='tests/data'
filelist = ['lsm-ifs.grb','onlytos-ipsl.nc','tas-ecearth.nc', 
            '2t-era5.nc','tos-fesom.nc', 'ua-ecearth.nc', 'mix-cesm.nc']#,'era5-mon.nc'] # the last is not available on github
#filelist = ['tos-fesom.nc','onlytos-ipsl.nc','tas-ecearth.nc'] 
#filelist = ['tas-ecearth.nc']
tfile = os.path.join(indir, 'r360x180.nc')

# method for remapping
methods = ['nn','con','bil']
#methods = ['con']
accesses = ['Dataset', 'DataArray']


# create an iterable dictionary, and clean cases where we know CDO does not work
defdict = {'methods': methods, 'accesses': accesses, 'extra': '', 'chunks': None}
base = {k: copy.deepcopy(defdict) for k in filelist}
if 'tos-fesom.nc' in filelist:
    base['tos-fesom.nc']['methods'].remove('bil')
if 'lsm-ifs.grb' in filelist:
    base['lsm-ifs.grb']['extra'] = '-setgridtype,regular'
    base['lsm-ifs.grb']['methods'].remove('bil')
    base['lsm-ifs.grb']['methods'].remove('con')
if 'mix-cesm.nc' in filelist:
    base['mix-cesm.nc']['accesses'].remove('DataArray')
if 'era5-mon.nc' in filelist:
    base['era5-mon.nc']['chunks'] = {'time': 12}
if 'ua-ecearth.nc' in filelist:
    base['ua-ecearth.nc']['chunks'] = {'plev': 3}

## Robustness test

This is to verify that the regridding is equal: this is done by comparing the output from CDO to the output obtained by SMM. 
The files to be checked are above. This is the same as the thing done in the tests.

In [2]:

for filein in filelist: 
    for method in base[filein]['methods']:
        for access in base[filein]['accesses']: 
            cc = check_cdo_regrid(os.path.join(indir,filein), tfile, method = method, access = access)#, extra = base[filein].get('extra'))
            print(filein + ': remap' + method + ' via ' + access + ' -> ' + str(cc))


lsm-ifs.grb: remapnn via Dataset -> True
lsm-ifs.grb: remapnn via DataArray -> True
onlytos-ipsl.nc: remapnn via Dataset -> True
onlytos-ipsl.nc: remapnn via DataArray -> True
onlytos-ipsl.nc: remapcon via Dataset -> True
onlytos-ipsl.nc: remapcon via DataArray -> True
onlytos-ipsl.nc: remapbil via Dataset -> True
onlytos-ipsl.nc: remapbil via DataArray -> True
tas-ecearth.nc: remapnn via Dataset -> True
tas-ecearth.nc: remapnn via DataArray -> True
tas-ecearth.nc: remapcon via Dataset -> True
tas-ecearth.nc: remapcon via DataArray -> True
tas-ecearth.nc: remapbil via Dataset -> True
tas-ecearth.nc: remapbil via DataArray -> True
2t-era5.nc: remapnn via Dataset -> True
2t-era5.nc: remapnn via DataArray -> True
2t-era5.nc: remapcon via Dataset -> True
2t-era5.nc: remapcon via DataArray -> True
2t-era5.nc: remapbil via Dataset -> True
2t-era5.nc: remapbil via DataArray -> True
tos-fesom.nc: remapnn via Dataset -> True
tos-fesom.nc: remapnn via DataArray -> True
tos-fesom.nc: remapcon via

From now we use only cases where we can use remapcon, i.e. we remove gaussiam reduced

## Full remapping 

Test the full remap (generation of the weight + applicaton) of CDO vs SMM. Still using conservative remapping. Results seems very much comparable!

In [2]:
# nrepetition for the check
nr = 5

if 'lsm-ifs.grb' in filelist:
    base.pop('lsm-ifs.grb')

# fast function to call the entire interpolation
def smm_remap(ifile, tfile):

    xfield = xr.open_mfdataset(ifile)
    wfield = cdo_generate_weights(ifile, tfile, method = 'con')
    interpolator = Regridder(weights=wfield)
    var = list(xfield.data_vars)[-1]
    rfield = interpolator.regrid(xfield)
    return(rfield)

data =[]
for filein in base.keys(): 

    one = timeit.timeit(lambda: cdo.remapcon(tfile, input = os.path.join(indir,filein), returnXDataset = True), number = nr)
    #print(filein + ': Exectime CDO Weight+Remap ' + str(one/nr))
    two = timeit.timeit(lambda: smm_remap(os.path.join(indir,filein), tfile), number = nr)
    #print(filein + ': Exectime SMM Weight+Remap ' + str(two/nr))
    data.append([one, two])

cnames = ['CDO (Weight+Remap)', 'SMM (Weight+Remap)']
df = pd.DataFrame(data, index = base.keys(), columns = cnames)
df.div(df[cnames[0]],axis =0)


caught signal <cdo.Cdo.__getattr__.<locals>.Operator object at 0x7fe995fca620> 2 <frame at 0x7fe8c829c040, file '/work/users/paolo/miniconda3/envs/smmregrid/lib/python3.10/selectors.py', line 416, code select>


# Remapping (with weights available)

This is the real goal of smmregrid. Here we test the computation of the remap when the weights are pre-computed, still using with conservative remapping. Considering that SMM does not have to write anything to disk, it is a few times faster. Running with Dataset implies a bit of overhead (20%). Masks have been integrated and create a small overhead when needed. Of course, loading the files implies a considerable slowdown.

In [3]:
if 'lsm-ifs.grb' in filelist:
    base.pop('lsm-ifs.grb')

data =[]
for filein in base.keys(): 
    print(filein)
    if filein in ['era5-mon.nc', 'era5-day.nc']: 
        nr = 2
    else :
        nr = 20

    # CDO
    wfile = cdo.gencon(tfile, input = os.path.join(indir,filein))
    ccdo = timeit.timeit(lambda: cdo.remap(tfile + ',' + wfile, input = os.path.join(indir,filein), returnXDataset = True).load(), number = nr)
    cdonoload = timeit.timeit(lambda: cdo.remap(tfile + ',' + wfile, input = os.path.join(indir,filein), returnXDataset = True), number = nr)
    #print(filein + ': Exectime CDO Remap ' + str(one/nr))

    # SMM: load field and weights, initialize regridder
    xfield = xr.open_mfdataset(os.path.join(indir,filein)).load()
    wfield = cdo_generate_weights(os.path.join(indir,filein), tfile, method = 'con').load()
    interpolator = Regridder(weights=wfield)
 
    # var as the one which have time and not have bnds, pick the first one
    myvar = [var for var in xfield.data_vars 
             if 'time' in xfield[var].dims and 'bnds' not in xfield[var].dims]
   
    # dataset infos
    nrecords = xfield[myvar[0]].shape
    nvars = len(myvar)


    sset =      timeit.timeit(lambda: interpolator.regrid(xfield).load(), number = nr)
    arr =       timeit.timeit(lambda: interpolator.regrid(xfield[myvar[0]]).load(), number = nr)
    arrnoload = timeit.timeit(lambda: interpolator.regrid(xfield[myvar[0]]), number = nr)
    #arrnomask = timeit.timeit(lambda: interpolator.regrid(xfield[myvar[0]], masked = False).load(), number = nr)
    setwrite =  timeit.timeit(lambda: interpolator.regrid(xfield).to_netcdf('test.nc'), number = nr)
    if os.path.isfile('test.nc'):
        os.remove('test.nc')
    arrwrite = timeit.timeit(lambda: interpolator.regrid(xfield[myvar[0]]).to_netcdf('test2.nc'), number = nr)
    if os.path.isfile('test2.nc'):
        os.remove('test2.nc')
    data.append([nvars, nrecords, ccdo, cdonoload, sset, arr, arrnoload, setwrite, arrwrite])


cnames = ['NVars', 'NRecords', 'CDO', 'CDO (NoLoad)',
          'SMM (Dataset)', 'SMM (DataArray)', 'SMM (DataArray+NoLoad)', 
          'SMM (Dataset+Write)', 'SMM (DataArray+Write)']
df = pd.DataFrame(data, index = base.keys(), columns = cnames)
final = pd.concat([df.iloc[:,0:2],df.iloc[:,2:].div(df[cnames[2]], axis=0)], join='outer', axis=1)
final


onlytos-ipsl.nc
tas-ecearth.nc
2t-era5.nc
tos-fesom.nc
ua-ecearth.nc
mix-cesm.nc


,NVars,NRecords,CDO,CDO (NoLoad),SMM (Dataset),SMM (DataArray),SMM (DataArray+NoLoad),SMM (Dataset+Write),SMM (DataArray+Write)
onlytos-ipsl.nc,1,"(12, 332, 362)",1.0,0.980259,0.356113,0.317013,0.077833,0.414487,0.397271
tas-ecearth.nc,1,"(12, 256, 512)",1.0,0.951850,0.342026,0.303791,0.077969,0.398138,0.389535
2t-era5.nc,1,"(12, 73, 144)",1.0,1.065076,0.234757,0.180491,0.045016,0.292226,0.278912
tos-fesom.nc,1,"(12, 126859)",1.0,0.961329,0.313225,0.311130,0.043078,0.366438,0.351693
ua-ecearth.nc,1,"(2, 19, 256, 512)",1.0,0.888290,0.505048,0.475789,0.147039,0.731295,0.718535
mix-cesm.nc,4,"(12, 192, 288)",1.0,0.878847,0.598981,0.172255,0.042516,0.922143,0.241714


Print to markdown so that we can copy paste it on the portal

In [3]:
print(final.to_markdown())

|                 |   NVars | NRecords          |   CDO |   CDO (NoLoad) |   SMM (Dataset) |   SMM (DataArray) |   SMM (DataArray+NoLoad) |   SMM (Dataset+Write) |   SMM (DataArray+Write) |
|:----------------|--------:|:------------------|------:|---------------:|----------------:|------------------:|-------------------------:|----------------------:|------------------------:|
| onlytos-ipsl.nc |       1 | (12, 332, 362)    |     1 |       0.924445 |        0.377513 |          0.347206 |                0.078164  |              0.429603 |                0.431012 |
| tas-ecearth.nc  |       1 | (12, 256, 512)    |     1 |       0.927699 |        0.410739 |          0.38098  |                0.0854462 |              0.468425 |                0.461594 |
| 2t-era5.nc      |       1 | (12, 73, 144)     |     1 |       0.924335 |        0.204548 |          0.160671 |                0.0439383 |              0.253002 |                0.248918 |
| tos-fesom.nc    |       1 | (12, 126859)      | 

## Weight generation

Test the different weights generation possibilities with CDO, tested with conservative remapping: the climtas code is way more efficient if files are already on the disk, since the call to CDO has to be done from file. CDO bindings have a minimum overhead to be considered

In [ ]:
# nrepetition for the check
nr = 5

if 'lsm-ifs.grb' in filelist:
    base.pop('lsm-ifs.grb')

# generate weights from file
data = []
for filein in base.keys(): 
 
    # open file
    xfield = xr.open_mfdataset(os.path.join(indir,filein))
    tfield = xr.open_mfdataset(tfile)

    # generate weights from file
    one = timeit.timeit(lambda: cdo_generate_weights(os.path.join(indir,filein), tfile, method = 'con'), number = nr)
    #print(filein + ': Exectime climtas from file ' + str(one/nr))
    # generate weights from xarray
    two = timeit.timeit(lambda: cdo_generate_weights(xfield, tfield, method = 'con'), number = nr)
    #print(filein + ': Exectime climtas from xarray ' + str(two/nr))
    # generatre weights with CDO bindings (from file)
    three = timeit.timeit(lambda: cdo.gencon(tfile, input = os.path.join(indir,filein), returnXDataset = True), number = nr)
    #print(filein + ': Exectime cdo from file ' + str(three/nr))
    data.append([three, one, two])

cnames = ['CDO bindings', 'CDO subprocess (from file)', 'CDO subprocess (from xarray)']
df = pd.DataFrame(data, index = base.keys(), columns = cnames)
df.div(df[cnames[0]],axis =0)


,CDO bindings,CDO subprocess (from file),CDO subprocess (from xarray)
onlytos-ipsl.nc,1.0,0.938895,1.025188
tas-ecearth.nc,1.0,0.882129,1.164144
2t-era5.nc,1.0,0.747073,0.852617
tos-fesom.nc,1.0,0.987944,1.377073
ua-ecearth.nc,1.0,0.891298,1.118633
mix-cesm.nc,1.0,0.796245,1.012632


: 